# Session 2 - Langchain Foundations

## Step-by-step

In [1]:
%pip install langchain

  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.4 MB/s eta 0:00:00a 0:00:01
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached jsonpointer-3.0.0-py2.py3-none-any.whl (7.6 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os
# load env file
from dotenv import load_dotenv
load_dotenv()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

True

In [4]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

# initialize the model
model = ChatOpenAI(model="gpt-4o")

/Users/ttran/PyCharmProjects/aiug-workship-vs/aiug-workshop/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

# system and human message templates
messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

# print message
print("message template")
print(messages)

# e.g., system message
print("\nparse message template")
print(messages[0].type)
print(messages[0].content)

# invoke the model with messages
print("\ninvoke model")
print(model.invoke(messages))

message template
[SystemMessage(content='Translate the following from English into Italian'), HumanMessage(content='hi!')]

parse message template
system
Translate the following from English into Italian

invoke model
content='Ciao!' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_bc2a86f5f5', 'finish_reason': 'stop', 'logprobs': None} id='run-9a16d1a8-ae7c-4947-b638-89a7d561691b-0' usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23}


In [7]:
from langchain_core.output_parsers import StrOutputParser

# initialize prebuilt output parser
parser = StrOutputParser()

In [8]:
# invoke model
result = model.invoke(messages)

In [9]:
# pass through string output parser
parser.invoke(result)

'Ciao!'

In [10]:
# chain together model and parser: output of the model is the input of the parser
chain = model | parser

In [7]:
# output is now a string
chain.invoke(messages)

'Ciao!'

In [8]:
from langchain_core.prompts import ChatPromptTemplate

In [9]:
# system template with placeholder language variable
system_template = "Translate the following into {language}:"

In [12]:
# construct chat prompt template with system template and user message placeholder
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

# print the prompt template
print(prompt_template.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translate the following into {language}:')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]


In [13]:
# pass in variables
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [14]:
# same messages structure as directly using system and human message templates
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [15]:
# chain all together
chain = prompt_template | model | parser

In [17]:
# invoke the chain with input variables
chain.invoke({"language": "italian", "text": "hi"})

'Ciao'

## Combined

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# define the model
model = ChatOpenAI(model="gpt-3.5-turbo-0125")

# define the prompt template
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

# define the output parser
parser = StrOutputParser()

# step wise result
result = prompt_template.invoke({"language": "italian", "text": "hi"})
result = model.invoke(result)
result = parser.invoke(result)
print("Step-wise:")
print(result)

# chain result
print("Chain:")
chain = prompt_template | model | parser
result = chain.invoke({"language": "italian", "text": "hi"})
print(result)

Step-wise:
Ciao
Chain:
Ciao
